In [40]:
# pip install python-dateutil

In [41]:
from kafka import KafkaConsumer
from pyspark.sql.types import StructType, StructField, TimestampType, IntegerType, StringType
from pyspark.ml.linalg import VectorUDT
from pyspark.sql.functions import to_timestamp

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("page_id", IntegerType(), True),
    StructField("url", StringType(), True),
    StructField("head", StringType(), True),
    StructField("author", StringType(), True),
    StructField("category", StringType(), True),
    StructField("date", StringType(), True),
    StructField("tags", StringType(), True),
    StructField("text", StringType(), True),
    StructField("summary", StringType(), True),
    StructField("hashtags", StringType(), True),
    StructField("keywords", StringType(), True),
])

# offset: subscribe from beginning of time
consumer = KafkaConsumer('news_from_step2', bootstrap_servers='127.0.0.1:9092', auto_offset_reset='earliest')


In [42]:
import pandas as pd

rows = []
while True:
    partition = ""
    for _, partition in consumer.poll(timeout_ms=5000, max_records=999999).items():
        for msg in partition:
            data = msg.value.decode('utf-8').split('\t')
            rows.append({
                "id": int(data[0]),
                "page_id": int(data[1]),
                "url": data[2],
                "head": data[3],
                "author": data[4],
                "category": data[5],
                "date": data[6],
                "tags": data[7],
                "text": data[8],
                "summary": data[9],
                "hashtags": data[10],
                "keywords": data[11],
            })
            print(f'\r{len(rows)}', end='')
    if len(partition) == 0:
        break

df = pd.DataFrame(rows)


1597

In [43]:
import requests
from elasticsearch import Elasticsearch
import csv
from dateutil import parser
import re 

In [44]:
es = Elasticsearch(hosts=['http://localhost:9200'])

index_name = 'news.'

if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
    print(f"Index '{index_name}' deleted successfully.")
else:
    print(f"Index '{index_name}' does not exist.")


Index 'news.' deleted successfully.


In [45]:
import requests
from elasticsearch import Elasticsearch
import csv
from dateutil import parser

# Persian month names mapping
PERSIAN_MONTHS = {
    "فروردین": 1,
    "اردیبهشت": 2,
    "خرداد": 3,
    "تیر": 4,
    "مرداد": 5,
    "شهریور": 6,
    "مهر": 7,
    "آبان": 8,
    "آذر": 9,
    "دی": 10,
    "بهمن": 11,
    "اسفند": 12
}

ELASTICSEARCH_ENDPOINT = 'http://localhost:9200'
INDEX_NAME = 'news.'
INDEX_MAPPING = {
    "mappings": {
        "properties": {
            "id": {"type": "integer"},
            "page_id": {"type": "integer"},
            "url": {"type": "keyword"},
            "head": {"type": "keyword"},
            "author": {"type": "keyword"},
            "category": {"type": "keyword"},
            "date": {"type": "keyword"}, 
            "tags": {"type": "keyword"},
            "text": {"type": "text"},
            "summary": {"type": "keyword"},
            "hashtags": {"type": "keyword"},
            "keywords": {"type": "keyword"}
        }
    }
}

# Create the index
response = requests.put(f'{ELASTICSEARCH_ENDPOINT}/{INDEX_NAME}', json=INDEX_MAPPING)
if response.ok:
    print("Index created successfully!")
else:
    print(f"Failed to create the index. {response.status_code}: {response.text}")

# Connect to Elasticsearch
es = Elasticsearch(hosts=['http://localhost:9200'])
index_name = 'news.'
file_path = 'news_202307170512.csvexport.csv'

# Function to process keywords
def process_keywords(keywords_str):
    keywords = re.findall(r"'(.*?)'", keywords_str)
    return keywords

# Function to convert Persian date to ISO 8601 format
def convert_persian_date_to_iso(date_str):
    # Remove any extra white spaces
    date_str = date_str.strip()

    # Split the Persian date string into day, month, and year
    day, month, year = date_str.split()[-3:]

    # Convert Persian month name to month number
    month_number = PERSIAN_MONTHS.get(month)

    # Create an ISO 8601 date string
    iso_date_str = f"{year}-{month_number:02d}-{int(day):02d}"
    return iso_date_str

# Read CSV file and perform bulk indexing
# with open(file_path, 'r', encoding='utf-8') as file:
#     reader = csv.DictReader(file)
#     my_body = []
#     for i, row in enumerate(reader, start=1):
#         my_body.append({"index": {"_index": index_name, "_id": i}})
#         my_body.append({
#             "id": int(row["id"]),
#             "page_id": int(row["page_id"]),
#             "url": row["url"],
#             "head": row["head"],
#             "category": row["category"],
#             "date": convert_persian_date_to_iso(row["date"]),  # Convert Persian date to ISO 8601 format
#             "tags": row["tags"],
#             "text": row["text"],
#             "summary": row["summary"],
#             "hashtags": row["hashtags"],
#             "keywords": process_keywords(row["keywords"])
#         })

my_body = []
for i, row in df.iterrows():
    # my_body.append({"index": {"_index": index_name, "_id": i+1}})
    t = {
        "id": int(row["id"]),
        "page_id": int(row["page_id"]),
        "url": row["url"],
        "head": row["head"],
        "author": row["author"],
        "category": row["category"],
        "date": convert_persian_date_to_iso(row["date"]),  # Convert Persian date to ISO 8601 format
        "tags": row["tags"],
        "text": row["text"],
        "summary": row["summary"],
        "hashtags": row["hashtags"],
        "keywords": process_keywords(row["keywords"])
    }
    my_body.append(t)

    # try:
    #     response = es.bulk(body=my_body)
    #     if response["errors"]:
    #         print("Bulk indexing failed. Details:")
    #         for item in response["items"]:
    #             if "index" in item and item["index"].get("error"):
    #                 print(item["index"]["error"])
    #     else:
    #         # print(f"{len(my_body) // 2} documents indexed successfully.")
    #         pass
    # except Exception as e:
    #     print(f"Bulk indexing failed: {e}")

    # es.bulk(body=my_body)
    # es.index(index=index_name, document=t)
    print('\r{}'.format(i), end='')

from elasticsearch.helpers import bulk
bulk(es, my_body, index=index_name)


Index created successfully!
1596

(1597, [])

In [46]:
from elasticsearch import Elasticsearch
from IPython.display import display

# Assuming you have Elasticsearch running on 'localhost' with the default port '9200'
es = Elasticsearch(hosts=['http://localhost:9200'])

# Query ب: Display the 10 recent posts that have been newly added to the system
query_1 = {
    "_source": ["text"],
    "query": {
        "match_all": {}
    },
    "size": 10,
    "sort": [
        {
            "date": {
                "order": "desc"
            }
        }
    ]
}

res_1 = es.search(index=index_name, body=query_1)
print("Query ب Result:")
for hit in res_1['hits']['hits']:
    display(hit['_source']['text'])



C:\Users\S\AppData\Local\Temp\ipykernel_63916\2549398139.py:23: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res_1 = es.search(index=index_name, body=query_1)


Query ب Result:


'شبکه اطلاع\u200cرسانی طلا ارز کریپتونیوز قیمتپپه ظرف ماه گذشته چشمگیر درصد افزایش یافتهاست آوریل فروردین زمانی کوین\u200cجکو ردیابی حرکات قیمت پپه پرداختهاست ارزش درصد افزایش یافتهاست قیمت  کوین سقف تاریخ دلار اردیبهشت ثبت رسیدهاست درصد کاهش یافتهاست میزان نوسان چشم\u200cانداز پپه کوین نامطمئن نامشخص کردهاست التکوین هفته گذشته مومنتوم صعود مطلوب دست آورده میزان ماه افت چشمگیر داشتهاست توجه این\u200cکه نهنگ پپه به\u200cعنوان وسیله کسب سود سریع کار برد توانست شاهد حرکات صعود قدرتمند بازار  کوین آینده بود اندیکاتورهای بازار پپه حاضر حاکی موقعیت نسبتا ضعیف  کوین پیش\u200cبینی قیمت بازار شاهد صعود دوباره سطوح بالا بود میانگین متحرک روزه التکوین حاضر میانگین روزه حرکت ظرف روز آینده رفت انتظار زمان وضعیت  کوین بهبود یافتهباشد شاخص قدرت نسبی پپه امروز صبح توانست عدد رسید حاضر پایین حرکت مسئله بالقوه نشان شاخص قدرت نسبی بازگشت قدرتمند احتمالا حوالی عدد کاهش سطح حمایت  کوین ظرف هفته گذشته بالا رفتهاست پیش\u200cبینی قیمت پپه حرکت صعود پایدار احتمالا سطوح پایین سقوط این\u200cکه هیچ\u200cگونه تحلی

'شبکه اطلاع\u200cرسانی طلا ارز کریپتوپوتیتو بایننس بزرگ صرافیارز دیجیتالجهان بیست چهارمین دوره ماهه توکن\u200cسوزی سوزاندن بایننس کوین انجام رساند رویداد ماه انجام شرکت بایننس کمک عرضه توکن بومی میلیون کاهش کمیاب نتیجه افزایش ارزش رسید کار تاثیری قیمت بایننس کوین داشتهباشد تازگی قابل\u200cتوجه قیمتریپل بایننس کوین جایگاه عنوان چهارمین ارز دیجیتال دنیا دست توکن بومی شرکت ریپل تقدیم فهرست بزرگ ارز دیجیتال ارزش بازار سایت کوین مارکت بایننس کوین رتبه پایین ریپل رتبه بالا ازیواس\u200cدی کوین جایگاه پنجم قرار اعلام صراف بایننس دوره توکن\u200cسوزی شرکت تقریا معادل میلیون دلار بایننس کوین سوزانده شدهاست اساس گزارش شرکت بایننس فرآیند بایننس کوین طریق برنامه\u200cیپایونیر برنسوزانده شدهاست شرکت بایننس ویژگی کمک مشتریانی ایجاد کردهاست دسترس دارایی بایننس کوین دست داد مدیر عامل شرکت بایننس چانگ پنگ ژائو مثبت ارزیاب فرایند اعلام الگوریتم عمو انجام شدهاست شرکت بایننس سال کاهش عرضه افزایش تقاضا معمول ماه مقدار معین توکن\u200cهای بومی آدرسی ارسال توانایی دسترس استراتژی فرایند نام توکن\u200cسوزی شهرت ی

'شبکه اطلاع\u200cرسانی طلا ارز کوین تلگراف داده موسسه گلسنود نشان بازار نزول می\u200cرسدبیت\u200cکوین گرفتار طول دوره نزول تاریخ ارز دیجیتال اساس گزارش\u200cها زمستان کریپتو تقریبا نوامبر قیمت بیت کوین سقف تاریخ حوالی دلار آغاز پادشاهارزهای دیجیتالهم\u200cاکنون حوالی دلار معامله درصد کاهش سقف تاریخ این\u200cکه دوره نزول بازار بیت کوین ادامه تحولات نشان پایان شدهباشیم دوره نزول بیت کوین پایان رسیده داده بازار نشان دومین دوره بزرگ قیمت بیت کوین طولانی فشار نزول باقی ماندهاست بیت کوین مراحل اولیه شکل\u200cگیری نقدینگی افزایش روز وضعیت نزول باقی ماند سرمایه\u200cگذار بیت\u200cکوین فشار نزول قرار زمان قیمت بیت کوین سقف سال شدت فاصله سقوط دوره نزول کریپتو سال نزول طولانی مد اواخر سال ادامه یافتهاست تاثیر عوامل بنیادین جمله اقدامات فدرال رزرو مهار تورم نتیجه افزایش نرخ بهره آغاز وحشت نگرانی سرمایه\u200cگذار سقوط شرک کریپتو جمله اروز کپیتال اف\u200cتی\u200cایکس وویجر فروپاشی استیبل کوین ترا یو\u200cاس\u200cدی اکوسیستم لونا دوره نزول دامن زد زمستان کریپتو سرد تاریک هش\u200cریت درخواست\u200cها ر

...

'شبکه اطلاع\u200cرسانی طلا ارزبر اساس داده وب\u200cسایت کوین مارکت قیمتریپل حوالی ساعت صبح روز چهارشنبه جولای تیر درصد افزایش روز گذشته دلار بزرگ روزه قیمت ریپل جولای تیر بودهاست لحظه نگارش مطلب حوالی ساعت ریپل اصلاح\u200cشده سنت بازگشته ریپل درصد افزایش قیمت دلار معامله افزایش قابل\u200cتوجه قیمت ریپل روز ارزش بازار دلار رساندهاست فهرست بزرگ\u200cترینارزهای دیجیتالاز ارزش بازار سایت کوین مارکت ریپل هم\u200cاکنون جایگاه قرار ارزش بازار ریپل بالا میلیارد بودهاست قیمت ریپل ظرف بیست ساعت گذشته محدوده دلار سقف دلار نوسان بودهاست التکوین ظرف هفته گذشته محدوده دلار معامله شدهاست اعلام را دادگاه جولای تیر مبنی آن\u200cکه ارز دیجیتال ریپل صراف آمریکایی کوین\u200cبیس فهرست قیمت ریپل شدت افزایش قیمت التکوین ظرف هفته گذشته درصد داشتهاست حجم معاملات بیست ساعته ریپل درصد افزایش رقم دلار رسیدهاست سویی ریپل قیمت کنون بالا سقف تاریخ دلار ژانویه ثبت رسیدهاست درصد فاصله توجه افزایش ملاحظه قیمت ریپل روز ارزش بهبیت کوین واتریوم درصد افزایش یافتهاست هم\u200cاکنون لحظه نگارش مطلب قیمت بیت کوین درصد افزایش ر

'شبکه اطلاع\u200cرسانی طلا ارز کوین\u200cپدیا بیت کوین دوره رکود اقتصاد سال عرصه گذاشت دهه گذشته بازار سهام همبستگی بودهاست میزان همبستگی بازار تازگی پایین سطح رسیدهاست زمانی اس\u200cاند\u200cپی چشم\u200cانداز نزول قرار بازار بیت کوین مسیر صعود بالعکس جیسون پیزینو تحلیلگر محبوب بازارارزهای دیجیتال پیش\u200cبینی کردهاست بیت کوین دیوار محکم دلار مقاومت سرسختی دست گریبان مقابل اس\u200cاند\u200cپی چشم\u200cانداز صعود مطلوب برخوردار خواهدبود تکنیکال تحلیلگر معتقد بیت کوین سال سقف تاریخ قیمت هزار دلار فاصله پیزینو پیش\u200cبینی قیمت بیت کوین سطح هزار دلار باقی خواهدماند هرچند تحلیلگر احتمال سقوط قیمت بیت کوین هزار دلار اندک میزان تقاضا سرمایه\u200cگذار نهادی بیت کوین افزایش\u200cیافته بعید شاهد سقوط عجیب غریب قیمت بود نمودار دوره صعود نزول انباشت آغاز تولد بیت کوین نشان تکرار مرتب چرخه نمودار نشان\u200cدهنده حقیقت قیمت بیت کوین وضوح چرخه تکرار پیروی هم\u200cاکنون اواخر دوره نزول اوایل دوره انباشت قرار خاطر پیزینو معتقد مقدمات بیت کوین سقف تاریخ سال آینده فراهم خواهدشد حاضر بهبود چشم\u200cاند

'شبکه اطلاع\u200cرسانی طلا ارز یوتودی الی مارتینز تحلیلگر معامله\u200cگر مشهور بازارارزهای دیجیتال انتشار پس تو اعلام سیگنال خرید نموداربیت کوین مشاهده کردهاست پست توضیح دادهاست پادشاه بازار ارز دیجیتال تحلیل تکنیکال حاضر کاشت انجام توانست جایگاه بالا سطوح دلار دست بیاورد اعتقاد الی مارتینز اندیکاتور تام دیمارک سیگنال خرید بیت کوین نشان دادهاست تحلیل نمودار بیت کوین الگو کندل دوجی معکوس برده نشان\u200cدهنده بازگشت نزول بیت\u200cکوین تحلیلگر معتقد بیت کوین توانست سقوط سطح دلار جلوگیری احتمالا مومنتوم آغاز حرکت صعود افزایش قیمت سطوح دلار دلار دست خواهدآورد گفته گزینه بیت کوین صعود ظرف ماه گذشته عوامل مختلف افزایش قیمت\u200cها بازار کریپتو شد جمله راه\u200cاندازی صراف ارز دیجیتال توسط غول وال\u200cاستریت درخواست راه\u200cاندازی صندوق معامله نقد بیت کوین بورس توسط شرکت بلک راک شرک پیروزی نسبیریپل دادگاه کمیسیون بورس اوراق بهادار آمریکا هیچ\u200cکدام توانست بیت کوین محدوده قیمت خارج بالا سطح دلار رساند اساس داده کوین مارکت هم\u200cاکنون لحظه نگارش مطلب حوالی ساعت روز چهارشنبه جولای تیر بیت 

'شبکه اطلاع\u200cرسانی طلا ارز کریپتونیوز قیمتسولانا به\u200cرغم سقوط درصدی روز هفته گذشته درصد روز گذشته درصد افزایش یافتهاست قیمت اینارز دیجیتالاز ابتدا سال درصد داشتهاست سولانا همراه التکوین\u200cها توانست سود توجه نتیجه مثبت پرونده حقوقیریپل کسب آن\u200cجایی حجم معاملات سولانا میلیارد دلار توانست افزایش قیمت آینده امیدوار بود نمودار قیمت سولانا نشان التکوین تجربه حرکت صعود مطلوب پایان هفته گذشته وضعیت اصلاح قیمت قرار اندیکاتورها حاکی احتمال کاهش قیمت شاخص قدرت نسبی سولانا گذراندن بیش\u200cتر روز آخر هفته نزدیک رسیدهاست احتمال شاخص قدرت نسبی سولانا سطوح قیمت کاهش احتمال میانگین متحرک روزه میانگین روزه بالا سطح رسیدهباشد معنی قرار سطوح پایین بازگشته قیمت سولانا همراه پایین بیاورد واقع قیمت سولانا حرکت سطح حمایتی جهش قیمت بازار هفته گذشته حرکت پیش\u200cبینی حرکت صعود تازه سولانا قیمت سطح دلار کاهش سولانا جنبه بنیادین وضعیت برد آن\u200cکه شبکه سولانا صدمات مشکلات گذاشت سال وضعیت تجربه شبکه سولانا حجم فروش توکن\u200cهای غیرمثلی رتبه قرار سولانا توانست ادامه هرچند پیش\u200cبینی قیمت روز 

'شبکه اطلاع\u200cرسانی طلا ارز کوین\u200cپدیا مایکل سیلر چهره برجسته دنیا فناوری طرفدارانبیت کوین چشم\u200cانداز مثبت پادشاهارزهای دیجیتالو پتانسیل مشکلات اقتصاد جهان صحبت اعتقاد مایکل سیلر به\u200cرغم بدبین چهره سن جامعه آینده بیت کوین علاقه فزاینده غول مالی دنیا جمله شرک بلک راک سیتادل دویچه بانک چارلز شواب فیدل به\u200cعنوان نشانه آشکار موضع جهانی ارز دیجیتال تلقی گفته سیلر بیت کوین شکل نوین پول دارایی سیستم اقتصاد پویا مدرن پرشتاب خلق پذیرش بیت کوین توسط غول مالی جهانی نشان بر سیستم مالی سن قرن بیستم سیستم مالی سن روز روز نقص معایب نشان منسوخ ادعا سیلر بیت کوین راه\u200cحل چال اقتصاد موسسات آن این\u200cکه عده بیت کوین رسمیت شناخته عنوان ابزار مناسب پرداخت سرمایه\u200cگذار پذیرفت مایکل سیلر کریپتو پذیرش گسترده توسط صندوق بازنشستگی شرک سرمایه\u200cگذار نهاد تجار عمو مختلف تاکید هرچند گفته مایکل سیلر مستلزم توسعه زیرساخت\u200cها توسط دول نهاد نظارتی مشارکت بانک مرکزی بین\u200cالمللی وجوه عمو اختیار تسهیل فرآیند پذیرش عمو کریپتو حائز اهمیت سیلر تصور نهاد دول جهان بیت کوین چالش کشید اظه

In [47]:
# Query ج: Display the number of posts based on the top 10 most used keywords
query_2 = {
    "size": 0,
    "aggs": {
        "top_keywords": {
            "terms": {
                "field": "keywords",
                "size": 10
            }
        }
    }
}

res_2 = es.search(index=index_name, body=query_2)
print("Query ج Result:")
for bucket in res_2['aggregations']['top_keywords']['buckets']:
    print(bucket['key'], ":", bucket['doc_count'])


Query ج Result:
دلار : 773
قیمت : 480
افزایش : 452
طلا : 333
کوین : 312
بازار : 300
روز : 247
کاهش : 223
بیت : 219
سطح : 199


C:\Users\S\AppData\Local\Temp\ipykernel_63916\3532010723.py:14: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res_2 = es.search(index=index_name, body=query_2)


In [48]:
# Query د: Display posts that include a specific word
query_3 = {
    "query": {
        "match": {
            "text": "کریپتو"
        }
    }
}

res_3 = es.search(index=index_name, body=query_3)
print("Query د Result:")
for hit in res_3['hits']['hits']:
    display(hit['_source']['text'])


Query د Result:


C:\Users\S\AppData\Local\Temp\ipykernel_63916\4206165114.py:10: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res_3 = es.search(index=index_name, body=query_3)


'شبکه اطلاع\u200cرسانی طلا ارز دیکریپت توجه بهبود قیمتبیت کوین پادشاه بازارارزهای دیجیتال ابتدا سال تحلیلگر سرمایه\u200cگذار نتیجه رسیدهباشند زمستان سرد طولانی کریپتو تقریبا پایان رسیدهاست قیمت ارز دیجیتال گذشت زمستان سرد طولانی توسعه نرم\u200cافزار صنعت کریپتو پایان نرسیدهباشد اساس تازه گزارش شرکت سرمایه\u200cگذاریالکتریک کپیتال توسعه\u200cدهندگان فعال پروژه منبع کریپتو سال گذشته درصد کاهش یافتهاست اساس آمار ماه ژوئن سال توسعه\u200cدهنده صنعت کریپتو فعالیت کرد سال گذشته نفر مایلز دویچر فعال حوزه دیفای تحلیلگر بازار کریپتو پس تو نوشت کاهش توسعه\u200cدهندگان صنعت کریپتو سال گذشته دلیل فرصت\u200cهای شغل حوزه بودهباشد پست تو نوشت بهبود قیمت\u200cها بازار ارز دیجیتال ژانویه سال میزان فعالیت توسعه\u200cدهندگان صنعت کاهش یافتهاست توسعه\u200cدهندگان صنعت کریپتو حوزه گرایش کرد روز پررونق شدهاست هوش مصنوع اعتقاد دویچر ناامیدی سرمایه\u200cگذار خطرپذیر خروج سرمایه صنعت کریپتو دلسرد توسعه\u200cدهندگان صنعت شدهباشد تحلیلگر پست نوشت خروج توسعه\u200cدهندگان صنعت کریپتو دلایل مختلف داشتهباشد جمله گرای

'شبکه اطلاع\u200cرسانی طلا ارز یوتودی اساس داده موسسه گلسنود توجه بازگشت سرمایه\u200cگذار استیبل کوین نشانه شکوفایی بازارارزهای دیجیتالدر آینده مشاهده موسسه گلسنود نتایج بررسی وضعیت عرضه استیبل کوین حاکی توجه خروج سرمایه بازار اساس نتایج بررسی\u200cها میزان تقاضای سرمایه\u200cگذار استیبل کوین افزایش\u200cیافته میزان ورود سرمایه بازار کریپتو بیش\u200cتر شدهاست نتایج تحلیل موسسه گلسنود نشان وضعیت نزول بازار کریپتو مشاهده تغییرات موجود عرضه ماهانه استیبل کوین بازه زمانی بلند یافت خروج سرمایه بازار استیبل کوین کاهش یافتهاست کاه عنوان نشانه بازگشت تقاضا سرمایه فضا بازار کریپتو تلقی میزان فعال بازار استیبل کوین نشانه امیدوارکننده پیشرفت بازار ارز دیجیتال خواهدبود استیبل کوین ارزش اعتبار دارا باثباتی دلار آمریکا متصل معمولا شرایط نوسان بازار عنوان پناهگاه امن محافظت دارا سرمایه\u200cگذار عمل استیبل کوین مه اکوسیستم کریپتو آن سرمایه\u200cگذار خروج بازار کریپتو توانست حفظ ارزش دارا اطمینان کسب افزایش میزان عرضه استیبل کوین نشان سرمایه\u200cگذار بازگرداندن دارا اکوسیستم کریپتو مسئله خودی نشانه ب

'شبکه اطلاع\u200cرسانی طلا ارز کوین تلگراف گفته\u200cیدیوید مارکوس چهره شناخته\u200cشده صنعت کریپتو بازاربیت\u200cکوین کریپتو سال پیامد سوءرفتارها سوءمدیریت\u200cهایبازیگران بدبازار نجات کرد دیوید مارکوس مدیرعامل بنیانگذار شرکتلایت\u200cاسپارک پس دسامبر نظرات درباره آینده بازار کریپتو امید معامله\u200cگر دنبال افزایش قیمت\u200cها حدود یاس کردهاست گفته مارکوس زمستان کریپتو احتمالا سال ادامه خواهدداشت ماه فاجعه فروپاشی صراف اف\u200cتی\u200cایکس گذشت عواقب هم\u200cچنان قیمت\u200cهایارزهای دیجیتالو نحوه عملکرد بازار کریپتو تاثیر منفی دیوید مارکوس سابقه فعالیت شرک متا پی\u200cپال اختیار بازیگر بدبازار کریپتو مسئول وضعیت کنون بازار پاسخگو سوءرفتارها سوءمدیریت\u200cشان پیامد رفتار پلیدانه غیرمسئولانه سال مد هم\u200cچنان صنعت کریپتو باقی خواهندماند این\u200cکه سخن نام اف\u200cتی\u200cایکس اشاره احتمالا بازیگر بدی رفتار پلیدانه غیرمسئولانه عملکرد ضعیف بازار گذشته آینده شد شخصسم بنکمن فرایدبوده مارکوس صریحا اعلام بازار کریپتو پایان سال احتمالا سال زمستان سرد تاریک خلاص نخواهدشد کارشناس صاحب\u200

...

'شبکه اطلاع\u200cرسانی طلا ارز کوین\u200cپدیا صعود درصدی قیمتبیت کوین هفته پادشاهارزهای دیجیتالبیش\u200cترین درصد افزایش هفتگی فوریه تجربه کردهاست بیت کوین حاضر بالا سطح دلار معامله بیت کوین شانزده روز سال گذشت صعود توجه نشان دادهاست قیمت رهبر بازار کریپتو بالا سطح زمان فروپاشی صراف اف\u200cتی\u200cایکس اوایل نوامبر رسیدهاست گای ترنر کارشناس سرمایه\u200cگذار مجری کانال یوتیوب پیش\u200cبینی کردهاست دوره رکود بازار ارز دیجیتال سال ادامه خواهدداشت گای ترنر عنوان متخصص برجسته حوزه کریپتو شناختهمی\u200cشود بازار سن دلیل کار ارتباط تنگاتنگ بازار ارز دیجیتال بازار سن عنوان کردهاست توجه چشم\u200cانداز نه\u200cچندان روشن بازار سهام بخش\u200cهای سن گای ترنر پیش\u200cبینی سرما زمستان طولانی\u200cمدت کریپتو افزایش خواهدیافت علی\u200cرغم آن\u200cکه قیمت بیت کوین دلار ژانویه حرکت دلچسب امیدوارکننده بالا دلار رسیدهاست ترنر پیش\u200cبینی رونق کنون بازار کریپتو احتمال دوام چند نخواهدداشت گای ترنر معتقد فعال بازار انتظار بازار سهام حدود افت شرایط آماده پیش\u200cبینی بازار کریپتو سقوط قیمت خواهدشد مد سقو

'شبکه اطلاع\u200cرسانی طلا ارز کوین تلگراف افزایش درصدی قیمت\u200cها بازار ارز دیجیتال روز منتهی سپتامبر شهریور سرمایه بازار کریپتو تریلیون دلار کردهاست حرکت صعود قابل\u200cتوجه پایان نزول بازار بازار روز گذشته نزول بودهاست ارزش بازار کریپتو سپتامبر شهریور سطح حمایت میلیارد دلار شدهبود وضعیت بازار سن روز بهبود درصدی بازار کریپتو روز شدهاست شاخص نزدک کامپوزیت غول فناوری سپتامبر شهریور درصد افزایش قیمت نفت وست تگزاس اینترمیدیت سپتامبر درصد افزایش داشتهاست داده همبستگی بازار سن بازار کریپتو اثبات پیگیری بررسی دقیق شرایط حاکم اقتصاد کلان پیش\u200cبینی تحولات آینده بازار کریپتو بااهمیت معیار همبستگی نزدک بیت کوین منفی مثبت متغیر معنی منفی نشان\u200cدهنده حرکت بازار جهت مخالف مثبت نشان\u200cدهنده حرکت هم\u200cسو نشان\u200cدهنده همبستگی بازار سرمایه\u200cگذار بازار سهام نگرانی منتظر نتایج سپتامبر شهریور فدرال رزرو آمریکا انتظار بانک مرکزی آمریکا نرخ بهره افزایش این\u200cکه بیش\u200cتر تحلیلگر بازار پیش\u200cبینی فدرال رزرو سومین متوالی نرخ بهره درصد افزایش سرمایه\u200cگذار امیدوارانه دنبال نش

'شبکه اطلاع\u200cرسانی طلا ارز کوین\u200cپدیا اساس گزارش تحقیق موسسه\u200cیکریپتوکوانتکه توسط وب\u200cسایتدبلاکمنتشر شدهاست میزان ذخایربیت کوین واتریوم موجود صرافی\u200cهایارز دیجیتالمستقر ایالات\u200cمتحده شدت کاهش\u200cیافته پایین سطح ژانویه رسیدهاست گزارش نشان میزان بیت\u200cکوین\u200cهایی صراف خارج آمریکا صراف بین\u200cالمللی مشتریان نزد نگه داشت صراف مستقر ایالات\u200cمتحده بیش\u200cتر شدهاست ذخایر کریپتو معمولا تسهیل انجام فعال معاملاتی بازار ارز دیجیتال عنوان معیار تشخیص میزان نقدینگی عمل کاهش ذخایر کریپتو احتمالا نشان\u200cدهنده حقیقت کاربر خارج دارا صراف آمریکایی کمیسیون بورس اوراق بهادار کمیسیون معاملات آتی کالا ایالات\u200cمتحده فشار بازار کریپتو روز روز افزایش کمیسیون بورس اوراق بهادار آمریکا هفته گذشته پرونده حقوق صراف بایننس کوین بیس تنظیم آن نقض مقررات اوراق بهادار متهم مقابل منطقه هنگ\u200cکنگ تازگی اتخاذ سیاست\u200cهایی شرک کریپتو مشتریان بیش\u200cتری جذب گفته موسسه کریپتوکوانت دولت ایالات\u200cمتحده نهاد قانون کشور اقدامات بازار کریپتو ادامه فعالیت کسب کار ارز دیجیتال

'شبکه اطلاع\u200cرسانی طلا ارز کوین\u200cپدیا سقوط قیمتارزهای دیجیتالدر ماه سال فروپاشی اف\u200cتی\u200cایکس نوامبر سال فعال بازار سرمایه\u200cگذار افتاد ارز دیجیتال مرگ نابودی مثلا تحلیلگر معامله\u200cگر بازار کریپتو سوال کلیدی پرسید قیمتاتریوم عنوان محبوب التکوین بازار سقف تاریخ جدید تجربه خواهدکرد سطوح فعلی باقی خواهدماند فشار بازار کریپتو سال گذشته حد بزرگ قو ارز دیجیتال یعنیبیت کوین ورشکستگی اف\u200cتی\u200cایکس سرعت ارزش دست گفته\u200cیگای ترنر تحلیلگر ارز دیجیتال شرکتکوین بیورو عوامل شروط خاص قیمت اتریوم بازار صعود دلار رسید اعتقاد گای ترنر ساخت اپلیکیشن بی\u200cرقیب توسط توسعه\u200cدهندگان اتریوم قیمت توکن بومی شبکه رقمی معتقد صنعت کریپتو فارغ صرفا بیت کوین اتریوم محصول اپلیکیشن جذاب کاربرد پاسخ پرسش اپلیکیشن جذاب بی\u200cرقیب محصول پلیکیشنی دانست بی\u200cدرنگ کار کرد حاضر ایده خاص درباره محصول شگفت\u200cانگیز جذاب صنعت کریپتو معتقد زود کارکرد مزایا اصل کریپتو همگان روشن خواهدشد همه مزایا کریپتو بهره\u200cمند پیش\u200cبینی آینده محصول اپلیکیشنی فوق\u200cالعاده جذاب کاربرد صنعت 

'شبکه اطلاع\u200cرسانی طلا ارز کریپتوپوتیتو اساس تحقیقات تازه موسسه\u200cیکوکوین درصد سرمایه\u200cگذار کریپتو کشور آلمان معتقدندارزهای دیجیتالمی\u200cتوانند تحقق برنامه بلندمدت\u200cشان انباشت ثروت آن کمک سرمایه\u200cگذار آل بازار کریپتو نفر کلاس دارایی بحبوحه آشفتگی\u200cها چال اقتصاد کنون عنوان ذخیره ارزش عمل ذخیره ارزش کالا دارایی گفتهمی\u200cشود معمول قدرت خرید آینده حفظ اساس نظرسنجی بیش\u200cتر سرمایه\u200cگذار آل حوزه کریپتو متعلق گروه جمع نسل وای هزاره نگاه دقیق\u200cترنتایج نظرسنجی نشان درصد سرمایه\u200cگذار ارز دیجیتال آلمان متعلق نسل وای نسل ایکس درصد سرمایه\u200cگذار آل تشکیل رتبه قرار دارا دیجیتال قاعدتا جو موضوع جالب آن نسل دنبال نوآور نسل کشور آلمان درصد سرمایه\u200cگذار فعال کریپتو تشکیل بیش\u200cتر آل کم\u200cتر ماه صنعت کریپتو آورد درصد آن ماه گذشته بازار ارز دیجیتال شد دلایل احتمال علاقه\u200cمند آل بازار کریپتو ماه گذشته احیا بازار ابتدا سال عنوان بیت کوین سال قیمت تقریبا دلار پایان رساند قیمت پادشاه ارز دیجیتال بالا سطح دلار قرار اساس داده وب\u200cسایت کوین مارکت هم

In [53]:
new_df = pd.DataFrame(my_body)

In [52]:
new_df.head()

,id,page_id,url,head,author,category,date,tags,text,summary,hashtags,keywords
0,32298,2172793,https://www.tgju.org/news/2630723/سناریوی-کاهش...,سناریوی کاهش نفت خام، ادامه‌دار,معصومه دانش,اخبار طلا و سکه,1402-03-22,"['بهای نفت', 'نفت خام', 'نفت برنت', 'نرخ بهره'...",شبکه اطلاع‌رسانی طلا ارز رویترز قیمتنفتروز دوش...,از آنجا که نگرانی در مورد موضوعاتی همچون تقاضا...,"['بهای نفت', 'نفت خام', 'نفت برنت', 'نرخ بهره'...","[نفت, کاهش, دلار, بشکه, افزایش]"
1,3,19,https://www.tgju.org/news/2638096/2-49,خرس‌های طلا در کمین!,معصومه دانش,اخبار طلا و سکه,1402-04-25,"['بهای طلا', 'بهای نقره', 'بازار طلا', 'نرخ به...",شبکه اطلاع‌رسانی طلا ارز کیتکونیوز تورم ضعیف ه...,گرچه طلا قیمت‌های طلا در هفته‌های اخیر رشد داش...,"['بهای طلا', 'بهای نقره', 'بازار طلا', 'نرخ به...","[نقره, طلا, فدرال, رزرو, بازار]"
2,4,21,https://www.tgju.org/news/2638081/چشم-انداز-بی...,چشم‌انداز بیت کوین تا پایان ماه جولای,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,1402-04-25,"['ریپل', 'بیت کوین', 'اتریوم', 'ارزهای دیجیتال...",شبکه اطلاع‌رسانی طلا ارز بیت‌کوین‌دات‌کام گذشت...,قیمت بیت‌کوین تحت تاثیر تحولات اخیر در پرونده‌...,"['ریپل', 'بیت کوین', 'اتریوم', 'ارزهای دیجیتال...","[سطح, قیمت, کوین, دلار, ماه]"
3,5,23,https://www.tgju.org/news/2638059/آیا-اکنون-وق...,آیا اکنون وقت خرید ریپل است؟,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,1402-04-25,"['ریپل', 'کمیسیون بورس و اوراق بهادار آمریکا',...",شبکه اطلاع‌رسانی طلا ارز کوین‌پدیا کمیسیون بور...,با انتشار خبری مبنی بر صدور یک حکم مطلوب برای ...,"['ریپل', 'کمیسیون بورس و اوراق بهادار آمریکا',...","[ریپل, قیمت, دلار, سطح, اوراق]"
4,30,50,https://www.tgju.org/news/2638164/تحلیل-و-پیش-...,تحلیل و پیش‌بینی قیمت ریپل؛ 25 تیر,امیر رضا مصطفایی,اخبار ارزهای دیجیتال,1402-04-25,"['بیت\u200cکوین', 'ریپل', 'ارزهای دیجیتال', 'ب...",شبکه اطلاع‌رسانی طلا ارز کوین‌پدیا قیمتبیت کوی...,پس از پیروزی نسبی ریپل در دادگاه،‌ قیمت آن به ...,"['بیت\u200cکوین', 'ریپل', 'ارزهای دیجیتال', 'ب...","[ریپل, قیمت, کوین, بازار, افزایش]"


In [54]:
from kafka import KafkaProducer
import os, random
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark.sql.functions import mean, col, desc, count, isnull, isnan, when, rank, sum
import pandas as pd
import matplotlib.pyplot as plt


producer = KafkaProducer(bootstrap_servers='127.0.0.1:9092')

rows = new_df.apply(lambda row: '\t'.join([str(value) for value in row]), axis=1)

for row in rows:
    producer.send('news_from_step3', value=row.encode('utf-8'))

producer.close()

print(f'Sent {len(rows)} lines in total.')


Sent 1597 lines in total.
